# 基本查询操作

## 文档的创建和实例

In [2]:
from mongoengine import connect, disconnect
from mongoengine import Document
from mongoengine import StringField, IntField
from mongo_config import TEST_DB1, TEST_DB2, TEST_DB3, HOST, PORT, USERNAME, PASSWORD
from mongoengine import BooleanField, DateTimeField, DictField, EmailField, FloatField, ListField, ReferenceField
from mongoengine import EmbeddedDocument, EmbeddedDocumentField, LazyReferenceField
from datetime import datetime
# 连接到已创建的数据库
connect(TEST_DB1, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB1, alias=TEST_DB1)
connect(TEST_DB2, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB2, alias=TEST_DB2)
connect(TEST_DB3, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB3, alias=TEST_DB3)

MongoClient(host=['192.168.2.172:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

### 建立查询要使用的全字段文档

In [3]:
class BookType(Document):
    type_name = StringField()
    meta = {"db_alias": TEST_DB1}

In [4]:
class BookAuthor(Document):
    name = StringField(unique=True)
    age = StringField()
    meta = {"db_alias": TEST_DB1}

In [9]:
class Seller(Document):
    name = StringField()
    phone = StringField(regex="^\d+")
    meta = {"db_alias": TEST_DB1}

In [65]:
class PublishingHouse(EmbeddedDocument):
    name =  StringField()  # unique=True,注意范围
    address =  StringField()

In [7]:
class BookAllField(Document):
    bookid = StringField(unique=True)
    name = StringField()
    ifnewbook = BooleanField()  # 是否为新上架图书
    shelves_date = DateTimeField()
    author = ReferenceField(BookAuthor)  # 使引用字段执行其他集合
    author_email = EmailField()
    price = FloatField()
    book_type = ReferenceField(BookType, dbref=True)  # 将引用ref文档
    publishing_house = EmbeddedDocumentField(PublishingHouse)
    seller = LazyReferenceField(Seller, dbref=True)
    tag = ListField()
    meta = {"db_alias": TEST_DB1}

In [16]:
from typing import List
def create_book(
        bookid: str, book_name: str, ifnewbook: bool, shelves_date: datetime, price: float, 
        tag: List[str], author_email: str, author_name: str,  author_age: str, 
        seller_name: str, seller_phone: str, book_type: str, publish_name: str, publish_address: str
    ):
    book = BookAllField(bookid=bookid, name = book_name, ifnewbook = ifnewbook, 
            shelves_date = shelves_date, price = price, tag = tag, author_email = author_email,
            publishing_house = PublishingHouse(name=publish_name, address=publish_address)
        )
    
    the_author = BookAuthor.objects(name=author_name).first()
    if the_author is None:
        the_author = BookAuthor(name=author_name, age=author_age).save()
    book.author = the_author
    
    the_seller = Seller.objects(name=seller_name).first()
    if the_seller is None:
        the_seller = Seller(name=seller_name, phone=seller_phone).save()
    book.seller = the_seller
    
    
    the_book_type = BookType.objects(type_name=book_type).first()
    if the_book_type is None:
        the_book_type = BookType(type_name=book_type).save()
    book.book_type = the_book_type
     
    book.save()

In [9]:
# 模拟数据
import random
from faker import Faker
fkdata = Faker(["ZH_cn"])

In [31]:
# 模拟数据
fkdata.name()
fkdata.address()
fkdata.phone_number()
fkdata.company()
fkdata.email()
fkdata.date()
fkdata.boolean()

'谢秀梅'

In [11]:
def random_create():
    book_name = "".join(fkdata.words(random.randint(1, 3)))
    create_book(
            bookid=fkdata.ssn(), book_name=book_name, ifnewbook=fkdata.boolean(), shelves_date=fkdata.date(), price=random.randint(100, 20000)/100, 
            tag=fkdata.words(random.randint(1, 3)), author_email=fkdata.email(), author_name=fkdata.name(),  author_age=random.choice(dynasty.split("、")), 
            seller_name=fkdata.company(), seller_phone=fkdata.phone_number(), book_type=fkdata.word(), publish_name=fkdata.company(), publish_address=fkdata.address()
        )

In [10]:
dynasty = "夏朝、商朝、西周、东周、秦朝、西楚、西汉、新朝、玄汉、东汉、三国、曹魏、蜀汉、孙吴、西晋、东晋、十六国、南朝、刘宋、南齐、南梁、南陈、北朝、北魏、东魏、北齐、西魏、北周、隋朝、唐朝、武周、唐朝中后期、五代、后梁、后唐、后晋、后汉、后周、十国、南吴、南唐、前属、后属、南汉、南楚、吴越、闽国、荆国、北汉、北宋、南宋、辽朝、西夏、金朝、元朝、明朝、清朝"

In [20]:
from tqdm import tqdm
for _ in tqdm(range(10000)):
    random_create()

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [05:44<00:00, 29.03it/s]


### 基本查询-过滤查询

The query may be filtered by calling the `QuerySet` object with field lookup keyword arguments. The keys in the keyword arguments correspond to fields on the Document you are querying:

#### 普通文档查询及获取引用文档

In [21]:
book1 = BookAllField.objects(name="支持这种").first()
book1.to_json(ensure_ascii=False)# to_json 使用json.dump的底层函数，因此可以使用json.dump配置

'{"_id": {"$oid": "64087d72cbd07ad5619989b5"}, "bookid": "511600193907182680", "name": "支持这种", "ifnewbook": true, "shelves_date": {"$date": 42854400000}, "author": {"$oid": "64087d72cbd07ad5619989b2"}, "author_email": "luojuan@example.org", "price": 55.46, "book_type": {"$ref": "book_type", "$id": {"$oid": "64087d72cbd07ad5619989b4"}}, "publishing_house": {"name": "戴硕电子科技有限公司", "address": "陕西省淑兰市合川潮州路T座 131033"}, "seller": {"$ref": "seller", "$id": {"$oid": "64087d72cbd07ad5619989b3"}}, "tag": ["包括", "其他"]}'

In [37]:
# 直接获取嵌入式文档
book1.publishing_house.to_json(ensure_ascii=False)

'{"name": "戴硕电子科技有限公司", "address": "陕西省淑兰市合川潮州路T座 131033"}'

In [39]:
# 直接获取引用字段，无dbref
book1.author.to_json(ensure_ascii=False)

'{"_id": {"$oid": "64087d72cbd07ad5619989b2"}, "name": "张彬", "age": "南朝"}'

In [42]:
# 直接获取引用字段，有dbref
book1.book_type.to_json(ensure_ascii=False)

'{"_id": {"$oid": "64087d72cbd07ad5619989b4"}, "type_name": "帮助"}'

In [41]:
# 直接获取引用字段;LazyReferenceField 在不使用 .fetch() 在此查询的环境下，不能获取到引用的文档。
book1.seller.fetch().to_json(ensure_ascii=False)

'{"_id": {"$oid": "64087d72cbd07ad5619989b3"}, "name": "兰金电子传媒有限公司", "phone": "18098307014"}'

In [66]:
books = BookAllField.objects(name="支持这种")
for book in books:
    print(book.to_json(ensure_ascii=False))  
# 实际上，由于 BookAllField 的 name 字段唯一，只能查询出一个文档

{"_id": {"$oid": "64087d72cbd07ad5619989b5"}, "bookid": "511600193907182680", "name": "支持这种", "ifnewbook": true, "shelves_date": {"$date": 42854400000}, "author": {"$oid": "64087d72cbd07ad5619989b2"}, "author_email": "luojuan@example.org", "price": 55.46, "book_type": {"$ref": "book_type", "$id": {"$oid": "64087d72cbd07ad5619989b4"}}, "publishing_house": {"name": "戴硕电子科技有限公司", "address": "陕西省淑兰市合川潮州路T座 131033"}, "seller": {"$ref": "seller", "$id": {"$oid": "64087d72cbd07ad5619989b3"}}, "tag": ["包括", "其他"]}


#### 嵌入式文档查询

嵌入式文档上的字段也可以通过使用双下划线代替对象属性访问语法中的点来使用字段查找语法来引用

In [67]:
# 嵌入式文档查询
books3 = BookAllField.objects(publishing_house__name="戴硕电子科技有限公司")
# book3 数据类型为 QuerySet，使用具体方法查询对应的API

In [69]:
books3.first().to_json(ensure_ascii=False)

'{"_id": {"$oid": "64087d72cbd07ad5619989b5"}, "bookid": "511600193907182680", "name": "支持这种", "ifnewbook": true, "shelves_date": {"$date": 42854400000}, "author": {"$oid": "64087d72cbd07ad5619989b2"}, "author_email": "luojuan@example.org", "price": 55.46, "book_type": {"$ref": "book_type", "$id": {"$oid": "64087d72cbd07ad5619989b4"}}, "publishing_house": {"name": "戴硕电子科技有限公司", "address": "陕西省淑兰市合川潮州路T座 131033"}, "seller": {"$ref": "seller", "$id": {"$oid": "64087d72cbd07ad5619989b3"}}, "tag": ["包括", "其他"]}'

In [ ]:
for book in books3:  # book的数据类型: BookAllField object
    print(book.name)# to_json 使用json.dump的底层函数，因此可以使用json.dump配置

In [ ]:
books3.to_json(ensure_ascii=False)

#### 原始查询

In [43]:
# 原始查询返回的 数据类型为 QuerySet，使用具体方法查询对应的API
books2 = BookAllField.objects(__raw__={"publishing_house.name": "戴硕电子科技有限公司"})
for book in books2:
    book.to_json(ensure_ascii=False)# to_json 使用json.dump的底层函数，因此可以使用json.dump配置

In [ ]:
books2.to_json(ensure_ascii=False)

#### 引用文档查询

In [89]:
# 先用关键字查询出被引用文档，再用被引用文档查询文档本身
filter_author = BookAuthor.objects(name="张彬")
books3 = BookAllField.objects(author=filter_author.first())

In [91]:
books3.to_json(ensure_ascii=False)

'[{"_id": {"$oid": "64087d72cbd07ad5619989b5"}, "bookid": "511600193907182680", "name": "支持这种", "ifnewbook": true, "shelves_date": {"$date": 42854400000}, "author": {"$oid": "64087d72cbd07ad5619989b2"}, "author_email": "luojuan@example.org", "price": 55.46, "book_type": {"$ref": "book_type", "$id": {"$oid": "64087d72cbd07ad5619989b4"}}, "publishing_house": {"name": "戴硕电子科技有限公司", "address": "陕西省淑兰市合川潮州路T座 131033"}, "seller": {"$ref": "seller", "$id": {"$oid": "64087d72cbd07ad5619989b3"}}, "tag": ["包括", "其他"]}, {"_id": {"$oid": "64087df3cbd07ad56199aad8"}, "bookid": "510801200403013639", "name": "环境使用觉得", "ifnewbook": false, "shelves_date": {"$date": 919382400000}, "author": {"$oid": "64087d72cbd07ad5619989b2"}, "author_email": "liuyong@example.org", "price": 53.57, "book_type": {"$ref": "book_type", "$id": {"$oid": "64087d73cbd07ad561998a36"}}, "publishing_house": {"name": "飞利信科技有限公司", "address": "福建省福州市城东李路Q座 243722"}, "seller": {"$ref": "seller", "$id": {"$oid": "64087d7ccbd07ad561998